In [1]:
import numpy as np
np.random.seed(1234)

In [2]:
class VBMC:
    def __init__(self, max_iter=100):
        self.max_iter = max_iter
        self.a_gamma0 = 1e-6
        self.b_gamma0 = 1e-6
        self.initial_r = 50
        self.prune_thres = 50
        
    def initialize(self, P, Y):
        self.m, self.n = Y.shape
        self.r = self.initial_r
        self.p = np.sum((P == 1) * 1.) / (self.m * self.n)
        U, S, VT = np.linalg.svd(Y, full_matrices=True)
        S_sqrt = np.sqrt(S)
        self.A = (U * S_sqrt.reshape(1, -1))[:, :self.r]
        self.B = (VT.T * S_sqrt.reshape(1, -1))[:, :self.r]
        scale = np.sqrt(np.mean(Y**2))
        self.Sigma_A = scale * np.tile(np.eye(self.r).reshape(self.r, self.r, 1), [1, 1, self.m])
        self.Sigma_B = scale * np.tile(np.eye(self.r).reshape(self.r, self.r, 1), [1, 1, self.m])
        self.gamma = np.ones(self.r) * scale
        self.beta = 1 / scale**2
        self.X = self.A @ self.B.T
        
    def run_one_iter(self, P, Y):
        Gamma = np.diag(self.gamma)
        # A
        for i in range(self.m):
            ob_index = (P[i] == 1)
            Bi = self.B[ob_index]
            self.Sigma_A[:, :, i] = np.linalg.inv(self.beta * (Bi.T @ Bi + np.sum(self.Sigma_B[:, :, ob_index], axis=2)) + Gamma)
            self.A[i, :] = self.beta * Y[[i], ob_index] @ Bi @ self.Sigma_A[:, :, i]
        # B
        for j in range(self.n):
            ob_index = (P[:, j] == 1)
            Aj = self.A[ob_index]
            self.Sigma_B[:, :, j] = np.linalg.inv(self.beta * (Aj.T @ Aj + np.sum(self.Sigma_A[:, :, ob_index], axis=2)) + Gamma)
            self.B[j, :] = self.beta * Y[ob_index, [j]].T @ Aj @ self.Sigma_B[:, :, j]
        # X
        X_new = self.A @ self.B.T
        self.X_diff = np.sum((X_new - self.X)**2) / np.sum(self.X**2)
        self.X = X_new
        # gamma
        self.gamma = (self.m + self.n + self.a_gamma0) \
                     / (self.b_gamma0 \
                        + np.diagonal(self.B.T @ self.B) + np.diagonal(np.sum(self.Sigma_B, axis=2)) \
                        + np.diagonal(self.A.T @ self.B) + np.diagonal(np.sum(self.Sigma_A, axis=2)))
        # beta
        err = np.sum((Y - P * self.X)**2)
        for i in range(self.m):
            ob_index = (P[i] == 1)
            err += np.trace(self.B[ob_index].T @ self.B[ob_index] @ self.Sigma_A[:, :, i])
            err += np.trace(self.A[[i]].T @ self.A[[i]] @ np.sum(self.Sigma_B[:, :, ob_index], axis=2))
            err += np.trace(self.Sigma_A[:, :, i] @ np.sum(self.Sigma_B[:, :, ob_index], axis=2))
        self.beta = (self.p * self.m * self.n) / err
        
        # prune
        max_gamma = np.min(self.gamma) * self.prune_thres
        if np.sum(self.gamma > max_gamma):
            index = (self.gamma <= max_gamma)
            self.A = self.A[:, index]
            self.B = self.B[:, index]
            self.gamma = self.gamma[index]
            self.Sigma_A = self.Sigma_A[index]
            self.Sigma_A = self.Sigma_A[:, index, :]
            self.Sigma_B = self.Sigma_B[index]
            self.Sigma_B = self.Sigma_B[:, index, :]
            self.r = len(self.gamma)

    def fit(self, P, Y):
        self.initialize(P, Y)
        for t in range(self.max_iter):
            self.run_one_iter(P, Y)
            if not (t + 1) % 10:
                print('The %d-th iteration finished.' % (t + 1))
                print('Rank=%d, X_difference=%.3fx1e-10' % (self.r, self.X_diff*1e10))
            if self.X_diff < 1e-9:
                print('***Converge at step %d***' % (t + 1))
                break
        self.r_hat = np.linalg.matrix_rank(self.X)
        print('Estimated Rank=%d' % self.r_hat)

In [3]:
r_list = [5, 10, 15, 20, 25, 30, 35, 40]
estimated_r_list = []
x_error_list = []
p = 0.2
for r in r_list:
    A = np.random.normal(0, 1, [300, r])
    B = np.random.normal(0, 1, [300, r])
    X = A @ B.T
    P = np.random.binomial(1, p, [300, 300])
    N = np.random.normal(0, 0.05, [300, 300])
    Y = X * P + N
    
    model_VBMC = VBMC()
    model_VBMC.fit(P, Y)
    estimated_r_list.append(model_VBMC.r_hat)
    x_error_list.append(np.mean(model_VBMC.X - X)**2)

The 10-th iteration finished.
Rank=5, X_difference=18.819x1e-10
***Converge at step 11***
Estimated Rank=5
The 10-th iteration finished.
Rank=10, X_difference=106454.972x1e-10
***Converge at step 18***
Estimated Rank=10
The 10-th iteration finished.
Rank=50, X_difference=11407706.077x1e-10
The 20-th iteration finished.
Rank=15, X_difference=11078.972x1e-10
The 30-th iteration finished.
Rank=15, X_difference=6.219x1e-10
***Converge at step 30***
Estimated Rank=15
The 10-th iteration finished.
Rank=50, X_difference=7232975.776x1e-10
The 20-th iteration finished.
Rank=50, X_difference=628918.599x1e-10
The 30-th iteration finished.
Rank=50, X_difference=205951.434x1e-10
The 40-th iteration finished.
Rank=50, X_difference=124326.559x1e-10
The 50-th iteration finished.
Rank=50, X_difference=107556.296x1e-10
The 60-th iteration finished.
Rank=50, X_difference=116743.090x1e-10
The 70-th iteration finished.
Rank=50, X_difference=142866.123x1e-10
The 80-th iteration finished.
Rank=50, X_differen

/home/bnmath/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


The 100-th iteration finished.
Rank=50, X_difference=nanx1e-10
Estimated Rank=2


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure()
plt.style.use('ggplot')
plt.plot(r_list, estimated_r_list)
plt.scatter(r_list, estimated_r_list)
plt.xlabel("True Ranks")
plt.ylabel("Estimated Ranks")
plt.xlim((0, 45))
plt.ylim((0, 45))
plt.savefig("VBMC_rank.pdf", dpi=300, quality=95)

plt.figure()
plt.style.use('ggplot')
plt.plot(r_list, [e * 1e5 for e in x_error_list])
plt.scatter(r_list, [e * 1e5 for e in x_error_list])
plt.xlabel("True Ranks")
plt.ylabel("Error x 1e-5")
plt.xlim((0, 45))
plt.savefig("VBMC_error.pdf", dpi=300, quality=95)